# Load the necessary modules

Zipline only works with an older version of pandas. It is necessary to downgrade to the version of pandas that is 0.22.0

In [1]:
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import zipline
from zipline.api import order, record, symbol, set_benchmark
from collections import OrderedDict
from datetime import datetime

# Load and Preprocess the data with zipline

In [2]:
# Store the different datasets
data = OrderedDict()
# Loads the data from the csv file
stocks  = ["AAPL", "AMZN", "BA", "FB", "GOOG", "MA", "MSFT", "NVDA", "UNH", "V"]

# Iterate to load stock data from csv
for stock in stocks:
    data[stock] = pd.read_csv("{}.csv".format(stock), index_col =0, parse_dates=['Date'])
    data[stock] = data[stock][["Open","High","Low","Close","Volume"]]
    data[stock] = data[stock].resample('1d').mean()
    data[stock].fillna(method='ffill', inplace=True)
    print(data[stock].head(10))

             Open   High    Low  Close      Volume
Date                                              
2013-01-02  38.54  38.99  38.48  38.85  15826400.0
2013-01-03  39.28  39.52  38.71  38.88  14937200.0
2013-01-04  38.90  39.30  38.69  39.19  10376000.0
2013-01-05  38.90  39.30  38.69  39.19  10376000.0
2013-01-06  38.90  39.30  38.69  39.19  10376000.0
2013-01-07  39.24  39.69  39.20  39.47  10242400.0
2013-01-08  39.41  39.84  39.25  39.84  11800400.0
2013-01-09  40.35  40.64  39.93  40.45  17552800.0
2013-01-10  40.51  40.69  39.82  40.13  16313600.0
2013-01-11  39.94  40.31  39.88  40.29   9578000.0
             Open   High    Low  Close      Volume
Date                                              
2013-01-02  38.54  38.99  38.48  38.85  15826400.0
2013-01-03  39.28  39.52  38.71  38.88  14937200.0
2013-01-04  38.90  39.30  38.69  39.19  10376000.0
2013-01-05  38.90  39.30  38.69  39.19  10376000.0
2013-01-06  38.90  39.30  38.69  39.19  10376000.0
2013-01-07  39.24  39.69  39.20

In [3]:
# Convert data to panel
StockPrice = pd.Panel(data)
StockPrice.minor_axis = ["Open","High","Low","Close","Volume"]
StockPrice.major_axis = StockPrice.major_axis.tz_localize(pytz.utc)
print(StockPrice)

<class 'pandas.core.panel.Panel'>
Dimensions: 10 (items) x 2187 (major_axis) x 5 (minor_axis)
Items axis: AAPL to V
Major_axis axis: 2013-01-02 00:00:00+00:00 to 2018-12-28 00:00:00+00:00
Minor_axis axis: Open to Volume


In [4]:
%load_ext zipline

In [6]:
# Define necessary two functions to run algorithm
def initialize(context):
    set_benchmark(symbol('AAPL'))
    
def handle_data(context, data):
    order(symbol('AAPL'),10)
    record(AAPL = data.current(symbol('AAPL'),'price'))
    

In [7]:
perf = zipline.run_algorithm(start=datetime(2013,1,2,0,0,0,0,pytz.utc),
                            end=datetime(2018,12,28,0,0,0,0,pytz.utc),
                            initialize=initialize,
                            capital_base=10000,
                            handle_data=handle_data,
                            data=StockPrice)

KeyError: "None of [['close']] are in the [minor_axis]"